# Tutorial 3: Application and Substitution

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-pure-passage-calculus/blob/main/notebooks/tutorial_03_application_and_substitution.ipynb)

---

## The Heart of Computation

In Year 722, Kelleth Mund wrote:

> *"A passage awaits its argument as a lock awaits its key. When they meet, transformation occurs."*

This tutorial explores **application**—the act of applying a passage to an argument—and **beta reduction**—the transformation that follows.

---

## Learning Objectives

By the end of this tutorial, you will:
1. Understand **application** of passages to arguments
2. Perform **beta reduction** (substitution)
3. Trace multi-step reductions
4. Recognize **redexes** (reducible expressions)

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

expressions = pd.read_csv(BASE_URL + "passage_calculus_expressions.csv")
reductions = pd.read_csv(BASE_URL + "passage_reductions.csv")

print(f"Loaded {len(expressions)} expressions")
print(f"Loaded {len(reductions)} reduction steps")

## Part 1: Application

**Application** is written by juxtaposition:

```
(f x)
```

This means: apply passage `f` to argument `x`.

When `f` is a lambda abstraction like `λy.body`, applying it to `x` substitutes `x` for `y` in `body`.

In [ ]:
# Simple application: identity applied to 5
identity = lambda x: x
result = identity(5)
print(f"(λx.x) 5 = {result}")

In [ ]:
# Application is left-associative
# (f x y) means ((f x) y)

K = lambda x: lambda y: x  # Returns first argument

# K applied to 'a' then to 'b'
result = K('a')('b')  # ((K 'a') 'b')
print(f"((λx.λy.x) a) b = {result}")

## Part 2: Beta Reduction

**Beta reduction** is the fundamental operation of the passage calculus.

The rule:
```
(λx.body) arg  →  body[arg/x]
```

Read as: "Replace all occurrences of x in body with arg."

### Example

```
(λx.x) a
  → x[a/x]
  → a
```

In [ ]:
# Let's look at reduction traces in Mund's data
reductions.head(10)

In [ ]:
# A specific reduction trace
trace_id = 'PR-003'  # K combinator trace
trace = reductions[reductions['reduction_id'] == trace_id].sort_values('step_number')
trace[['step_number', 'expression_after', 'rule_applied', 'notes']]

In [ ]:
# Let's trace this manually
print("Tracing: ((λx.λy.x) a) b")
print()

# Step 1: Apply (λx.λy.x) to a
print("Step 1: ((λx.λy.x) a) b")
print("         ↓ Beta reduce: (λy.x)[a/x] = λy.a")
print("       ((λy.a) b)")
print()

# Step 2: Apply (λy.a) to b
print("Step 2: (λy.a) b")
print("         ↓ Beta reduce: a[b/y] = a  (y doesn't appear in body!)")
print("        a")
print()
print("Result: a (the first argument)")

## Part 3: Reduction Strategies

When an expression has multiple redexes (reducible expressions), we must choose which to reduce first.

### Leftmost-Outermost (Normal Order)
Always reduce the leftmost, outermost redex first.

### Leftmost-Innermost (Applicative Order)  
Always reduce arguments before applying.

Most programming languages use applicative order. The lambda calculus traditionally uses normal order because it guarantees finding a result if one exists.

In [ ]:
# Look at reduction strategies in the data
reductions['rule_applied'].value_counts()

In [ ]:
# Find traces that show different strategies
normal_order = reductions[reductions['notes'].str.contains('Normal|Leftmost', case=False, na=False)]
normal_order[['reduction_id', 'initial_expression', 'notes']].drop_duplicates('reduction_id')

In [ ]:
# Why strategy matters: K discards its second argument
# (K a) Omega should return a, but...

# In normal order: reduce (K a) first
# (K a) Omega → (λy.a) Omega → a  ✓ Terminates!

# In applicative order: try to reduce Omega first
# But Omega = (λx.x x)(λx.x x) → (λx.x x)(λx.x x) → ... Never terminates!

# Let's find this example in the data
k_omega = reductions[reductions['initial_expression'].str.contains('K.*Omega|Omega.*K', case=False, na=False, regex=True)]
k_omega[['reduction_id', 'initial_expression', 'terminates', 'notes']].drop_duplicates('reduction_id')

## Part 4: Multi-Step Reductions

Let's trace a more complex reduction: adding Church numerals.

In [ ]:
# Find the addition trace
addition_trace = reductions[reductions['reduction_id'] == 'PR-013'].sort_values('step_number')
print("Tracing: 1 + 1 using Church numerals\n")

for _, row in addition_trace.iterrows():
    print(f"Step {row['step_number']}: {row['expression_after']}")
    print(f"         Rule: {row['rule_applied']}")
    print(f"         Note: {row['notes']}")
    print()

In [ ]:
# Verify with Python
# Church numerals
zero = lambda f: lambda x: x
one = lambda f: lambda x: f(x)
two = lambda f: lambda x: f(f(x))

# Addition
plus = lambda m: lambda n: lambda f: lambda x: m(f)(n(f)(x))

# Add 1 + 1
result = plus(one)(one)

# To verify, apply to successor and 0
succ = lambda n: n + 1
print(f"1 + 1 = {result(succ)(0)}")  # Should be 2

## Part 5: Statistics on Reductions

In [ ]:
# How many steps do reductions typically take?
final_steps = reductions[reductions['is_normal_form'] == True]
final_steps['total_steps'].describe()

In [ ]:
# Distribution of reduction lengths
terminating = final_steps[final_steps['terminates'] == True]

plt.figure(figsize=(10, 5))
terminating['total_steps'].hist(bins=20, color='steelblue', edgecolor='white')
plt.title('Distribution of Reduction Lengths (Terminating Expressions)')
plt.xlabel('Number of Steps')
plt.ylabel('Count')
plt.show()

In [ ]:
# Who traced these reductions?
reductions['traced_by'].value_counts()

In [ ]:
# What about non-terminating reductions?
non_terminating = reductions[reductions['terminates'] == False]
non_terminating_unique = non_terminating.drop_duplicates('reduction_id')
print(f"Non-terminating expressions: {len(non_terminating_unique)}")
non_terminating_unique[['reduction_id', 'initial_expression', 'notes']].head()

## Part 6: Implementing Beta Reduction

In [ ]:
# A simple AST representation
class Var:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return self.name
    def __eq__(self, other):
        return isinstance(other, Var) and self.name == other.name

class Lam:
    def __init__(self, var, body):
        self.var = var  # string
        self.body = body  # expression
    def __repr__(self):
        return f"(λ{self.var}.{self.body})"

class App:
    def __init__(self, func, arg):
        self.func = func
        self.arg = arg
    def __repr__(self):
        return f"({self.func} {self.arg})"

# Test
identity = Lam('x', Var('x'))
print(identity)

In [ ]:
def substitute(expr, var, replacement):
    """
    Substitute replacement for var in expr.
    
    This is a simplified version that doesn't handle capture-avoidance.
    """
    if isinstance(expr, Var):
        if expr.name == var:
            return replacement
        else:
            return expr
    elif isinstance(expr, Lam):
        if expr.var == var:
            # Variable is shadowed, don't substitute in body
            return expr
        else:
            return Lam(expr.var, substitute(expr.body, var, replacement))
    elif isinstance(expr, App):
        return App(
            substitute(expr.func, var, replacement),
            substitute(expr.arg, var, replacement)
        )
    return expr

# Test substitution
# (λx.x)[a/x] should remain λx.x (x is bound)
result = substitute(Lam('x', Var('x')), 'x', Var('a'))
print(f"Substituting a for x in λx.x: {result}")

# (λy.x)[a/x] should become λy.a
result = substitute(Lam('y', Var('x')), 'x', Var('a'))
print(f"Substituting a for x in λy.x: {result}")

In [ ]:
def beta_reduce_once(expr):
    """
    Perform one beta reduction step if possible.
    Returns (reduced_expr, did_reduce).
    """
    if isinstance(expr, Var):
        return expr, False
    
    elif isinstance(expr, Lam):
        new_body, reduced = beta_reduce_once(expr.body)
        if reduced:
            return Lam(expr.var, new_body), True
        return expr, False
    
    elif isinstance(expr, App):
        # Check if this is a redex: (λx.body) arg
        if isinstance(expr.func, Lam):
            result = substitute(expr.func.body, expr.func.var, expr.arg)
            return result, True
        
        # Try reducing the function
        new_func, reduced = beta_reduce_once(expr.func)
        if reduced:
            return App(new_func, expr.arg), True
        
        # Try reducing the argument
        new_arg, reduced = beta_reduce_once(expr.arg)
        if reduced:
            return App(expr.func, new_arg), True
        
        return expr, False
    
    return expr, False

# Test: reduce (λx.x) a
expr = App(Lam('x', Var('x')), Var('a'))
print(f"Before: {expr}")
result, _ = beta_reduce_once(expr)
print(f"After:  {result}")

In [ ]:
def reduce_fully(expr, max_steps=100):
    """
    Reduce expression to normal form (if it terminates).
    """
    steps = [expr]
    for _ in range(max_steps):
        new_expr, reduced = beta_reduce_once(expr)
        if not reduced:
            break
        expr = new_expr
        steps.append(expr)
    return steps

# Test: K a b = a
K_body = Lam('y', Var('x'))
K = Lam('x', K_body)
expr = App(App(K, Var('a')), Var('b'))

print("Reducing ((λx.λy.x) a) b:")
for i, step in enumerate(reduce_fully(expr)):
    print(f"  Step {i}: {step}")

## Exercises

### Exercise 1: Manual Reduction

Reduce each expression by hand, then verify with the data:

1. `(λx.x) (λy.y)`
2. `((λx.λy.y) a) b`
3. `((λf.λx.f x) (λy.y)) z`

In [ ]:
# Exercise 1 workspace

# 1. (λx.x) (λy.y)
#    → ?

# 2. ((λx.λy.y) a) b
#    → ?
#    → ?

# 3. ((λf.λx.f x) (λy.y)) z
#    → ?
#    → ?
#    → ?

### Exercise 2: Finding Traces

Use the reductions dataset to find:
1. The longest terminating reduction trace
2. All reductions traced by Jorell Vance
3. Reductions involving Church numerals

In [ ]:
# Exercise 2 workspace

# 1. Longest terminating trace
# Hint: filter for terminates==True, then find max total_steps

# 2. Reductions by Vance
# Hint: filter traced_by

# 3. Church numeral reductions
# Hint: search notes or initial_expression for 'Church'

### Exercise 3: Build an Expression

Using our AST classes (Var, Lam, App), build and reduce:

1. The identity applied to itself: `(λx.x)(λx.x)`
2. Church numeral two applied to a function and base: `((λf.λx.f(f x)) succ) 0`

In [ ]:
# Exercise 3 workspace

# 1. (λx.x)(λx.x)
# expr = App(..., ...)
# print(reduce_fully(expr))

# 2. Church two applied
# two = Lam('f', Lam('x', App(Var('f'), App(Var('f'), Var('x')))))
# ...

## Summary

In this tutorial, we learned:

1. **Application** `(f x)` applies passage `f` to argument `x`
2. **Beta reduction** `(λx.body) arg → body[arg/x]` substitutes the argument
3. **Reduction strategies** determine which redex to reduce first
4. **Normal order** guarantees finding a result if one exists
5. Multi-step reductions eventually reach **normal form** (if they terminate)

### Next Tutorial

In Tutorial 4, we'll explore **simplification rules**—when is an expression fully reduced? What is normal form?